In [2]:
#Import libraries in python3 kernel
import pandas as pd
import seaborn as sns
import glob
import os
import sys
from pathlib import Path
#!conda install --yes --prefix {sys.prefix} boto
import boto
import shutil
#!conda install --yes --prefix {sys.prefix} tqdm
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as colors
from matplotlib.colors import LogNorm
import numpy as np
import skbio
#import fastcluster #this package makes skbio run faster clustermaps but can be tricky with missing values from pairwise comparisons
from functools import reduce
#!conda install --yes --prefix {sys.prefix} biopython
from Bio import SeqIO
from Bio.SeqIO.FastaIO import SimpleFastaParser
from Bio.SeqUtils import GC
from biom import load_table
from biom.table import Table
from collections import defaultdict
from collections import Counter
import statistics
import itertools as it
from scipy import stats
from matplotlib.ticker import FormatStrFormatter
import matplotlib.colors as mcolors
from qiime2 import Artifact
import tempfile
import zipfile
import yaml

#!conda install --yes --prefix {sys.prefix} -c etetoolkit ete3 
#!conda install -c bioconda seqkit
#pip install ete3
#conda install -c anaconda pyqt
#from ete3 import Tree, TreeStyle
%matplotlib inline

#Import libraries
#from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
#from matplotlib_venn import venn3, venn3_circles
from matplotlib import pyplot as plt

In [8]:
# Special thanks to Alex Manuele https://github.com/alexmanuele
def consolidate_tables(community):
    
    if community == "16S":
        comm_id, comm = '16S', '02-PROKs'
        table_list = glob.glob('{0}/*/DADA2/table.qza'.format('/Users/Diana/Documents/escuela/phd/ch2/2014_trims/'+comm+'/all_trims'))
        print("Found all 16S tables")
    if community == "18S":
        comm_id, comm = '18S','02-EUKs'
        table_list = glob.glob('{0}/*/DADA2/table.qza'.format('/Users/Diana/Documents/escuela/phd/ch2/2014_trims/'+comm+'/all_trims'))
        print("Found all tables")
        
    dataframes = []  
    for table_path in table_list:
        with tempfile.TemporaryDirectory() as tempdir:
            #load table, dump contents to tempdir
            table = Artifact.load(table_path)
            #Make sure the tables are all FeatureFrequency type
            assert str(table.type) == 'FeatureTable[Frequency]', "{0}: Expected FeatureTable[Frequency], got {1}".format(table_path, table.type)
            Artifact.extract(table_path, tempdir)
            #get the provenance form the tempdir and format it for DF
            prov = '{0}/{1}/provenance/'.format(tempdir, table.uuid)
            action = yaml.load(open("{0}action/action.yaml".format(prov), 'r'), Loader=yaml.BaseLoader)
            paramlist = action['action']['parameters']
            paramlist.append({'table_uuid': "{}".format(table.uuid)})
            paramdict = {}
            for record in paramlist:
                paramdict.update(record)

            # Get the data into a dataframe
              #Biom data
            df = table.view(pd.DataFrame).unstack().reset_index()
            df.columns = ['feature_id', 'sample_name', 'feature_frequency']
            df['table_uuid'] = ["{}".format(table.uuid)] * df.shape[0]
              #param data
            pdf = pd.DataFrame.from_records([paramdict])
              #merge params into main df
            df = df.merge(pdf, on='table_uuid')
            

            #I like having these columns as the last three. Makes it more readable
            cols = df.columns.tolist()
            reorder = ['sample_name', 'feature_id', 'feature_frequency']
            for val in reorder:
                cols.append(cols.pop(cols.index(val)))
            df = df[cols]
            df['table_path'] = [table_path] * df.shape[0]
            dataframes.append(df)
            
            #Adding table_id, forward and reverse trim columns
            df['table_id'] = str(table_path.split('/')[-3]) #add a table_id column
            df['forward_trim'], df['reverse_trim'] = df['table_id'].str.split('R', 1).str
            df['forward_trim'] = df['forward_trim'].map(lambda x: x.lstrip('F'))
            df["forward_trim"] = pd.to_numeric(df["forward_trim"])
            df["reverse_trim"] = pd.to_numeric(df["reverse_trim"])

    #Stick all the dataframes together
    #outputfile="merged_all_tables.tsv"
    df = pd.concat(dataframes)
    df['sampleid'] = df['sample_name'].str.split('-S').str.get(0)
    df['sampleid'] = df['sampleid'].str.replace('-', '.')
    #df.to_csv(comm+'/merged_all_tables.tsv', sep='\t', index=False)
    print("Successfully saved all tables.")
    return df

In [13]:
def merge_metadata(df):
    all_md = pd.read_csv('/Users/Diana/Documents/escuela/phd/ch2/bedfordbasinTS/allmetadata.csv')

    tables = df[['sampleid', 'feature_id', 'feature_frequency', 'table_id', 'forward_trim', 'reverse_trim']].copy()
    tables = tables[tables.feature_frequency != 0]

    all_md['sampleid'] = all_md['sampleid'].str.replace('_', '.')
    merged = pd.merge(tables,all_md, on='sampleid', how='left') #all_md is the metadata file
    print('Set up metadata ...')
    
    #merged.to_csv(comm+'/merged_asvs_metadata.tsv', sep = '\t')
    print('Saved merged_asvs_metadata.tsv')
    
    return merged

In [18]:
def rename_move_all_taxonomies(comm):
    dr = comm+'/intermediate_files/all_trims'
    if not os.path.isdir(dr):
        for root, dirs, files in os.walk(dr): #rename all taxonomy.tsv by their trimlengths
            for file in files:
                if file == "taxonomy.tsv":
                    spl = root.split("/"); newname = spl[-6]; sup = ("/").join(spl[:-6])
                    shutil.copy(root+"/"+file, sup+"/"+newname+".tsv");# shutil.rmtree(root)
        files = glob.glob('{0}F*R*.tsv'.format(dr))
        os.mkdir(dr)
        for file in files:
            shutil.move(file, dr) #puts all tsvs in new directory with correct names
    print('Renamed all taxonomies.')

In [19]:
def rename_move_statistics(comm):
    dr = comm+'/intermediate_files/'
    statdir = comm+'/intermediate_files/all_statistics'
    if not os.path.isdir(statdir): #check if the folder exists
        for root, dirs, files in os.walk(dr): #rename all stats.tsv by their trimlengths
            for file in files:
                if file == "stats.tsv":
                    spl = root.split("/"); newname = spl[-5]; sup = ("/").join(spl[:-6])
                    shutil.copy(root+"/"+file, sup+"/"+newname+".tsv");# shutil.rmtree(root)
        files = glob.glob('{0}F*R*.tsv'.format(dr))
        os.mkdir(statdir)
        for file in files:
            shutil.move(file, statdir) #puts all tsvs in new directory with correct names
    print('Renamed all statistics.')

In [9]:
df = consolidate_tables('16S')

Found all 16S tables


/var/folders/kv/yq6jygk116l27q2bwnk25k0w0000gn/T/ipykernel_65532/1727549619.py:52: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df['forward_trim'], df['reverse_trim'] = df['table_id'].str.split('R', 1).str
/var/folders/kv/yq6jygk116l27q2bwnk25k0w0000gn/T/ipykernel_65532/1727549619.py:52: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df['forward_trim'], df['reverse_trim'] = df['table_id'].str.split('R', 1).str
/var/folders/kv/yq6jygk116l27q2bwnk25k0w0000gn/T/ipykernel_65532/1727549619.py:52: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df['forward_trim'], df['reverse_trim'] = df['table_id'].str.split('R', 1).str
/var/folders/kv/yq6jygk116l27q2bwnk25k0w0000gn/T/ipykernel_65532/1727549619.py:52: FutureWarning: Columnar iteration over characters will be deprecated in

Successfully saved all tables.


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49754208 entries, 0 to 941779
Data columns (total 24 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   table_uuid                      object 
 1   trunc_len_f                     object 
 2   trunc_len_r                     object 
 3   trim_left_f                     object 
 4   trim_left_r                     object 
 5   max_ee_f                        object 
 6   max_ee_r                        object 
 7   trunc_q                         object 
 8   min_overlap                     object 
 9   pooling_method                  object 
 10  chimera_method                  object 
 11  min_fold_parent_over_abundance  object 
 12  allow_one_off                   object 
 13  n_threads                       object 
 14  n_reads_learn                   object 
 15  hashed_feature_ids              object 
 16  sample_name                     object 
 17  feature_id                 

In [14]:
merged = merge_metadata(df)

Set up metadata ...
Saved merged_asvs_metadata.tsv


In [15]:
merged

,sampleid,feature_id,feature_frequency,table_id,forward_trim,reverse_trim,Unnamed: 0,year,date,weekn,...,HPLC_CHLC3,HPLC_DIADINOX,HPLC_DIATOX,HPLC_FUCOX,HPLC_HEX19,HPLC_PERID,HPLC_ZEA,POC,PON,time_string
0,BB14.10A,df29deb0aff9e983f503e7471e71f4ec,2458.0,F250R270,250,270,56,2014,05-Mar,10,...,0.000,0.450,0.0,1.878,0.0,0.000,0.0,NaN,NaN,2014-03-05
1,BB14.10A,df29deb0aff9e983f503e7471e71f4ec,2458.0,F250R270,250,270,57,2014,05-Mar,10,...,0.000,0.450,0.0,1.878,0.0,0.000,0.0,NaN,NaN,2014-03-05
2,BB14.10B,df29deb0aff9e983f503e7471e71f4ec,3222.0,F250R270,250,270,58,2014,05-Mar,10,...,0.174,0.373,0.0,1.704,0.0,0.000,0.0,305.3,47.9,2014-03-05
3,BB14.10B,df29deb0aff9e983f503e7471e71f4ec,3222.0,F250R270,250,270,59,2014,05-Mar,10,...,0.174,0.373,0.0,1.704,0.0,0.000,0.0,305.3,47.9,2014-03-05
4,BB14.10C,df29deb0aff9e983f503e7471e71f4ec,3040.0,F250R270,250,270,60,2014,05-Mar,10,...,0.000,0.119,0.0,1.209,0.0,0.000,0.0,NaN,NaN,2014-03-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3378168,BB14.36C,a29294dffae12d1bba4795eaa1d1ac76,1.0,F250R260,250,260,267,2014,03-Sep,36,...,0.000,0.000,0.0,0.374,0.0,0.000,0.0,NaN,NaN,2014-09-03
3378169,BB14.46C,fda17e00db8510f67c3ca9e21c004527,1.0,F250R260,250,260,346,2014,12-Nov,46,...,0.000,0.000,0.0,0.572,0.0,0.326,0.0,NaN,NaN,2014-11-12
3378170,BB14.46C,fda17e00db8510f67c3ca9e21c004527,1.0,F250R260,250,260,347,2014,12-Nov,46,...,0.000,0.000,0.0,0.572,0.0,0.326,0.0,NaN,NaN,2014-11-12
3378171,BB14.50A,f79783d82d8a20f6e76500bb29e66f01,1.0,F250R260,250,260,374,2014,10-Dec,50,...,0.000,0.000,0.0,0.226,0.0,0.000,0.0,NaN,NaN,2014-12-10
